### The benchmark of SARCOS dataset

The performance of different algorithms on this dataset can be found in: https://github.com/Kaixhin/SARCOS

#### Ridge regression with polynomial(3) + fourier([0.1:10:20]) feature: 1.1988,  
NN feature not working.

In [2]:
from aslearn.data.get_data import robot_inv_data
import numpy as np
import torch as th
device = "cuda" if th.cuda.is_available() else "cpu"

Loss = th.nn.MSELoss()
X_train, Y_train, X_val, Y_val, X_test, Y_test = robot_inv_data()

In [2]:
X_train.shape

(40000, 21)

In [2]:
X, Y = np.concatenate([X_train, X_val], axis=0), np.concatenate([Y_train, Y_val], axis=0)
from aslearn.feature.global_features import PolynomialFT
from aslearn.feature.bellcurve import BellCurve
from aslearn.feature.global_features import FourierFT
from aslearn.parametric.ridge import RidgeReg

f1 = PolynomialFT(degree=3) # sklearn.preprocessing.PolynomialFeature is much faster when len(X) is very large !
# f2 = FourierFT(degree=[1.])
X = X.astype(np.float64)
X_test = X_test.astype(np.float64)
X_f = f1(X)#np.concatenate([f1(X),f2(X)], axis=1)
X_test_f = f1(X_test)#np.concatenate([f1(X_test),f2(X_test)], axis=1)
print("feature dim: ", X_f.shape[1])

feature dim:  2024


In [4]:
X_f_t, Y_t, X_test_f_t, Y_t_test = \
th.from_numpy(X_f).to(device).double(), th.from_numpy(Y).to(device).double(),\
th.from_numpy(X_test_f).to(device).double(), th.from_numpy(Y_test).to(device).double()
rr = RidgeReg().fit(X_f_t, Y_t)
pred = rr.predict(X_test_f_t)
mse = Loss(pred, Y_t_test)
print(mse)

100%|███████████████████████████████████████████| 10/10 [02:01<00:00, 12.11s/it]


tensor(1.2818, device='cuda:0', dtype=torch.float64)


VRVM: 1.3459, maximum likelihood linear reg: 1.2800

In [6]:
%%time
from aslearn.parametric.mllr import MLLR
blr = MLLR().fit(X_f_t, Y_t, info_level=1)
pred = blr.predict(X_test_f_t)
mse = Loss(pred, Y_t_test)
print(mse)

Interation: 1 Curr_tol:  2765.6447
Interation: 2 Curr_tol:  0.0114
Interation: 3 Curr_tol:  0.0049
Interation: 4 Curr_tol:  0.0082
Interation: 5 Curr_tol:  0.0041
Interation: 6 Curr_tol:  0.0156
Interation: 7 Curr_tol:  0.0045
Interation: 8 Curr_tol:  0.0033
Interation: 9 Curr_tol:  0.0078
Interation: 10 Curr_tol:  0.0118
Interation: 11 Curr_tol:  0.0049
Interation: 12 Curr_tol:  0.0061
Interation: 13 Curr_tol:  0.0070
Interation: 14 Curr_tol:  0.0033
Interation: 15 Curr_tol:  0.0076
Interation: 16 Curr_tol:  0.0104
Interation: 17 Curr_tol:  0.0032
Interation: 18 Curr_tol:  0.0037
Interation: 19 Curr_tol:  0.0082
Interation: 20 Curr_tol:  0.0046
tensor(1.2800e+00, device='cuda:0', dtype=torch.float64)
CPU times: user 8.33 s, sys: 4.39 ms, total: 8.33 s
Wall time: 8.32 s


In [3]:
X_train, Y_train, X_val, Y_val, X_test, Y_test =\
    th.from_numpy(X_train).to(device).double(), th.from_numpy(Y_train).to(device).double(), th.from_numpy(X_val).to(device).double(), \
    th.from_numpy(Y_val).to(device).double(), th.from_numpy(X_test).to(device).double(), th.from_numpy(Y_test).to(device).double()
    
from torch.optim import Adam
from torch.nn import MSELoss
from aslearn.parametric.mlp import MLP

#### Neural network: 1.469

In [6]:
param = {"layer":4, "nodes":[21, 500, 500, 7], "actfunc":["ReLU", "ReLU", None], "batch":128, "lr":1e-3, "decay":0.}
net = MLP(param).to(device).double()
print(net)
Loss = MSELoss()
parameters = MLP.setParams(net, param["decay"])
optimizer = Adam(parameters, lr=param["lr"])
for _ in range(100):
    for i in range(len(X_train)//param["batch"]):
        optimizer.zero_grad()
        index = th.randperm(len(X_train))
        curr_index = index[i*param["batch"]:(i+1)*param["batch"]]
        X_b = X_train[curr_index]
        Y_b = Y_train[curr_index]
        pred = net(X_b)
        L = Loss(pred, Y_b)
        L.backward()
        optimizer.step()
    with th.no_grad():
        pred_val = net(X_val)
        L_val = Loss(pred_val, Y_val)
        print("Curr validation loss: ", L_val)
net.eval()
pred_test = net(X_test)
print(Loss(pred_test, Y_test))

MLP(
  (net): Sequential(
    (0): Linear(in_features=21, out_features=500, bias=True)
    (1): ReLU()
    (2): Linear(in_features=500, out_features=500, bias=True)
    (3): ReLU()
    (4): Linear(in_features=500, out_features=7, bias=True)
  )
)
Curr validation loss:  tensor(23.1734, device='cuda:0', dtype=torch.float64)
Curr validation loss:  tensor(18.8395, device='cuda:0', dtype=torch.float64)
Curr validation loss:  tensor(14.8275, device='cuda:0', dtype=torch.float64)
Curr validation loss:  tensor(13.2116, device='cuda:0', dtype=torch.float64)
Curr validation loss:  tensor(12.2317, device='cuda:0', dtype=torch.float64)
Curr validation loss:  tensor(10.8721, device='cuda:0', dtype=torch.float64)
Curr validation loss:  tensor(9.9622, device='cuda:0', dtype=torch.float64)
Curr validation loss:  tensor(10.9913, device='cuda:0', dtype=torch.float64)
Curr validation loss:  tensor(9.5698, device='cuda:0', dtype=torch.float64)
Curr validation loss:  tensor(9.7632, device='cuda:0', dtype=t

#### subsample GPR with hyperparameters optimization: 2.887

In [4]:
from aslearn.nonparametric.gpr import ExactGPR
from aslearn.kernel.kernels import White, RBF

In [16]:
X_test, Y_test = X_test.double(), Y_test.double()
perm = th.randperm(X_train.size(0))
idx = perm[:3000]
X_train_, Y_train_ = X_train[idx].double(), Y_train[idx].double()

In [6]:
from aslearn.parametric.mllr import MLLR
mp = MLLR().fit(X_train, Y_train) # the linear mean prior

In [7]:
l = th.ones((21,7)) * 20.
c1 = th.ones((7,)) * 0.5
c2 = th.ones((7,)) * 0.5

In [17]:
kernel = White(c=c1, dim_in=21, dim_out=7) + RBF(l=l, c=c2, dim_in=21, dim_out=7)
gpr = ExactGPR(kernel=kernel).fit(X_train_, Y_train_, mean_prior=None, info_level=1, call_opt=False)

Final loss: inf


In [13]:
with th.no_grad():
    params = list(gpr.kernel.parameters())
    c1 = params[0].data
    c2 = params[1].data[:7]
    l = params[1].data[7:].reshape(21,7)

In [18]:
pred = gpr.predict(X_test)
print(Loss(pred, Y_test))

tensor(4.3950e+00, device='cuda:0', dtype=torch.float64)


## Titsias' variational EM GPR: m=250, 2.8788

In [ ]:
from aslearn.nonparametric.vigpr import VariationalEMSparseGPR
from aslearn.kernel.kernels import White, RBF

l = np.ones([21, 1]) * 1.
sigma = np.ones([1,]) * 1.
c = np.ones([1,]) * 0.1

white_kernel = White(c=c, dim_in=21, dim_out=1)
kernel = RBF(sigma=sigma, l=l, dim_in=21, dim_out=1)

gpr = VariationalEMSparseGPR(kernel=kernel, white_kernle=white_kernel)
ind = gpr.fit(X_train, Y_train, m=20, subsetNum=15)

Current mean elbo: -718055.41:  85%|██████████▏ | 17/20 [07:03<01:21, 27.27s/it]

In [7]:
pred = gpr.predict(X_test)
print(Loss(pred, Y_test))

del gpr, pred, kernel
th.cuda.empty_cache()

tensor(9.2787, device='cuda:0', dtype=torch.float64,
       grad_fn=<MseLossBackward0>)
